In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import gaussian_kde
from statsmodels.tsa.seasonal import seasonal_decompose
from plotly.subplots import make_subplots
import numpy as np
from time import sleep

In [2]:
def write_image(fig, fname, show=False):
    if show:
        fig.show()

    # create random figure to load math js
    fig2 = px.scatter(x=[0, 1], y=[0, 1])
    fig2.write_image(fname)
    sleep(1)

    fig.write_image(fname)

In [3]:
load = pd.read_csv('../../../data/load/load3.csv.gz', compression='gzip')
pv = pd.read_csv('../../../data/renewable/renewable3.csv.gz', compression='gzip')

In [4]:
for vals, name in zip([-load['0'].dropna(), pv['0'].dropna()], ['load', 'pv']):
    if name == 'pv':
        vals.index = pd.date_range(start='2024-01-01', periods=len(vals), freq='h')
        
        # resample to daily frequency
        vals = vals.resample('D').mean()

    q1 = vals.quantile(0.25)
    q2 = vals.quantile(0.50)
    q3 = vals.quantile(0.75)

    kde = gaussian_kde(vals)
    x_vals = np.linspace(vals.min(), vals.max(), 500)
    y_vals = kde(x_vals)

    # Create figure and KDE line
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x_vals, y=y_vals, mode='lines', name='KDE'))

    # no outliers
    fig.add_shape(
        type="rect",
        x0=q1, x1=q3,
        y0=0, y1=max(y_vals)*1.1,
        fillcolor="rgba(255, 0, 0, 0.4)",
        opacity=0.2,
        layer="below",
        line_width=0,
    )

    # Add vertical dashed lines for quartiles
    for q, label in zip([q1, q2, q3], ['Q1', 'Median', 'Q3']):
        fig.add_shape(
            type="line",
            x0=q, x1=q,
            y0=0, y1=max(y_vals),
            line=dict(color="black", width=1, dash="dash"),
        )

        # Add label annotation
        fig.add_annotation(
            x=q, y=max(y_vals),
            text=label,
            yshift=20,
            showarrow=False,
            yanchor="top",
            font=dict(size=14)
        )

    # Layout settings
    fig.update_layout(
        height=500,
        width=700,
        margin=dict(l=20, r=20, t=20, b=20),
        xaxis_title='kWh',
        yaxis_title='Density',
        yaxis=dict(
            range=[0, max(y_vals) * 1.05],
            # no 0 as tick
            tickvals=[0.002 + i * 0.002 for i in range(6)],
            ticktext=[f'{0.002 + i * 0.002:.3f}' for i in range(6)],
        ),
        xaxis=dict(
            zeroline=True,
        ),
    )

    write_image(fig, f'{name}_kde.pdf', show=True)
    write_image(fig, f'{name}_kde.png')

In [5]:
load.shape[0] / 24

365.0

In [6]:
for vals, name in zip([-load['0'].dropna(), pv['0'].dropna()], ['load', 'pv']):
    vals.index = pd.date_range(start='2024-01-01', periods=len(vals), freq='h')

    # resample to daily frequency
    vals = vals.resample('D').mean()

    # Seasonal decomposition (requires no missing dates and regular frequency)
    decomp = seasonal_decompose(vals, model='additive', period=7)  # e.g., weekly seasonality

    # Plot with Plotly
    fig = make_subplots(rows=4, cols=1, subplot_titles=['Observed', 'Trend', 'Seasonal', 'Residual'], vertical_spacing=0.09)

    fig.add_trace(go.Scatter(x=vals.index, y=decomp.observed, name='Observed'), row=1, col=1)
    fig.add_trace(go.Scatter(x=vals.index, y=decomp.trend, name='Trend'), row=2, col=1)
    fig.add_trace(go.Scatter(x=vals.index, y=decomp.seasonal, name='Seasonal'), row=3, col=1)
    fig.add_trace(go.Scatter(x=vals.index, y=decomp.resid, name='Residual'), row=4, col=1)

    fig.update_layout(
        height=750,
        width=1000,
        margin=dict(l=20, r=20, t=20, b=20),
        showlegend=False,
    )

    # Graph range
    valid_trend = decomp.trend.dropna()
    x_start = valid_trend.index[0]
    x_end = valid_trend.index[-1]

    # Set x-axis range for all subplots
    fig.update_xaxes(range=[x_start, x_end], row=1, col=1)
    fig.update_xaxes(range=[x_start, x_end], row=2, col=1)
    fig.update_xaxes(range=[x_start, x_end], row=3, col=1)
    fig.update_xaxes(range=[x_start, x_end], row=4, col=1)

    write_image(fig, f'{name}_decomp.pdf', show=True)
    write_image(fig, f'{name}_decomp.png')

## LOAD
🔹 Observed (Raw Daily Load)
The observed data shows a clear upward trend in the first half (January to mid-February), followed by a slight decline starting around late February.

There is daily variability (likely due to consumer behavior, weather, etc.), but it's fairly consistent in amplitude.

🔹 Trend
The trend increases steadily from January through mid-February, peaking around late February, then declines gradually.

This could indicate:

Increasing energy demand in early winter (or post-holiday activity).

Then possibly reduced heating needs or energy-saving measures kicking in later.

🔹 Seasonal
You have a strong and regular weekly pattern — this is typical for residential or commercial energy loads.

The seasonality shows higher demand on specific weekdays and lower on others, likely related to work-week vs weekend consumption patterns.

🔹 Residual
The residual component (what's left after removing trend and seasonal parts) is centered around zero, with no obvious pattern — this is a good sign, meaning your decomposition model captures most systematic patterns.

Some spikes exist, possibly due to outliers or anomalies (e.g., sudden weather changes or holidays).

📌 Interpretation Summary:
The decomposition validates the presence of weekly seasonality and a mid-term trend, both of which are important if you're planning forecasting or optimization (e.g., for energy trading or demand-response).

## PV
🔹 Observed (Raw PV Output)
The observed PV production shows:

A rising trend from early January to around mid-February.

Followed by a gradual decline through March.

Daily variation is visible, but less erratic than the load.

This is consistent with solar irradiance patterns — production increases as days get longer, and then possibly decreases due to weather or angle changes.

🔹 Trend
Clear bell-shaped pattern: starts low, peaks mid-February, and declines in March.

This trend may reflect:

Changing daylight duration (shorter in January, longer toward March).

Possibly increased cloud cover or seasonal weather factors reducing sunlight after the peak.

🔹 Seasonal
There’s a strong weekly cycle in PV production:

Peaks once a week (likely weekends or cloudless days).

This is a bit unusual since solar production is typically daily rather than weekly — this may suggest:

Systematic weekly weather patterns (e.g., clearer skies on certain days).

Or even data aggregation quirks.

If the source data is daily averages, it's worth double-checking whether the raw 15-min PV data truly supports weekly seasonality or if daily seasonality would be more appropriate for finer resolution.

🔹 Residual
Residuals are moderate in amplitude and fairly centered around zero.

A few spikes suggest outliers or noise, perhaps caused by unexpected cloud cover or sensor issues.

📌 Overall Interpretation:
PV production is mostly smooth and predictable, with a seasonal peak in mid-February.